
# Muhammad Ghulamzaki


# Preparation

Preparation biasa dilakukan untuk mempersiapkan data sebelum masuk dalam tahap pemodelan. <br>
Berikut adalah tahapan yang akan dilalui pada data `SC_HW1_bank_data.csv` sebelum tahap pemodelan :
1. Import Library
2. Input Dataset
3. Preprocessing
4. Train-Test Split

## Import Library

In [32]:
import pandas as pd
import numpy as np

## Input Dataset

In [33]:
#Membaca data dan memasukkannya ke dalam bentuk Pandas Dataframe
df = pd.read_csv('https://raw.githubusercontent.com/Rietaros/kampus_merdeka/main/SC_HW1_bank_data.csv')

In [34]:
#Jalankan code untuk mengecek nama kolom yang tersedia
df.columns

Index(['RowNumber', 'CustomerId', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [35]:
#Hilangkan kolom yang dirasa tidak relevan dengan model (contoh: ID). None dapat diisi dengan nama-nama kolom yang akan digunakan.
#Contoh df = df[['X1','X2', 'Y']].copy()

#START CODE
#Menghilangkan kolom 'RowNumber' dan memilih kolom-kolom yang relevan
df = df[['Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']].copy()
df.columns

Index(['Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

## Preprocessing

In [36]:
#Lakukan One-Hot Encoder untuk data categorical, dengan fungsi pandas get_dummies

#START CODE
df_encoded = pd.get_dummies(df, columns=['Geography', 'Gender'])
df_encoded.head()

,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [37]:
#Pisahkan mana X (feature) dengan Y,
#Y adalah kolom "Exited"

#START CODE
#Pisahkan X (fitur) dan Y (target)
X = df_encoded.drop(columns=['Exited'])  # Menghilangkan kolom 'Exited' dari fitur X
y = df_encoded['Exited']  # Kolom 'Exited' menjadi target Y

In [38]:
X.head()

,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,42,2,0.00,1,1,1,101348.88,1,0,0,1,0
1,41,1,83807.86,1,0,1,112542.58,0,0,1,1,0
2,42,8,159660.80,3,1,0,113931.57,1,0,0,1,0
3,39,1,0.00,2,0,0,93826.63,1,0,0,1,0
4,43,2,125510.82,1,1,1,79084.10,0,0,1,1,0


In [39]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [40]:
#Lakukan Scaler dan/atau Noermalisasi jika diperlukan
from sklearn.preprocessing import MinMaxScaler

#START CODE
scaler = MinMaxScaler()
X_transform = scaler.fit_transform(X)

In [41]:
#Ini digunakan jika dilakukan scaler/Normalisas. Jika tidak, code ini bisa dilewat dan diganti dengan code yang ada di dalam komen
X_transform = pd.DataFrame(X_transform, columns = X.columns)
#X_transform = X.copy()

## Train-Test Split

In [42]:
#Split menjadi train dan test dengan test_size 25%
#Tidak perlu mengubah code ini

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_transform,y,test_size = 0.25,random_state = 123)

# Modeling

## Model1

### Soal :
Model 1: RandomForestClassifier
Model ini menggunakan algoritma Random Forest, yaitu sebuah ensemble learning method yang menggabungkan beberapa decision trees untuk membuat prediksi. Setiap decision tree dihasilkan dari dataset yang berbeda dan memilih hasil paling populer sebagai prediksi akhir.

In [43]:
#Pilih salah satu metode Machine Leaarning
#Model Machine Learning dapat dipanggil terlebih dahulu melalui library yang digunakan. Gunakan library scikit learn seperti pada contoh

#Contoh pemanggilan library dan penggunaannya dalam model

#from sklearn.linear_model import LogisticRegression
#model1 = LogisticRegression()
#params = { "tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}

from sklearn.ensemble import RandomForestClassifier

#START CODE
model1 = RandomForestClassifier(random_state=123)
params = {
    'n_estimators': [50, 100, 150],  # Jumlah pohon dalam ensemble
    'max_depth': [None, 10, 20, 30],  # Kedalaman maksimum tiap pohon
    'min_samples_split': [2, 5, 10]  # Jumlah sampel minimum untuk membagi node internal
}

#END CODE

#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= model1,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs = 10, # core cpu yang digunakan
             cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

{'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 150}

In [44]:
#lakukan evaluasi dengan beberapa maetric di bawah ini
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92      1983
           1       0.78      0.44      0.56       517

    accuracy                           0.86      2500
   macro avg       0.82      0.70      0.74      2500
weighted avg       0.85      0.86      0.84      2500


[[1919   64]
 [ 289  228]]

0.8588


## Model2
### Soal :
Model 2: Support Vector Machine (SVC)
Model ini menggunakan Support Vector Machine (SVM), yaitu metode machine learning untuk tugas klasifikasi atau regresi. SVM mencari hyperplane terbaik yang memisahkan data berlabel ke dalam kelas-kelas yang berbeda.

In [16]:
from sklearn.svm import SVC

#START CODE
model2 = SVC()
params = {
    'C': [0.1, 1, 10],  # Parameter penalti
    'kernel': ['linear', 'rbf'],  # Fungsi kernel yang digunakan
    'gamma': ['scale', 'auto'],  # Koefisien kernel untuk 'rbf', 'poly' dan 'sigmoid'
}
#ENDCODE

# Melakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= model2,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs = 10, # Jumlah core CPU yang digunakan
             cv = 10 # 10-fold cross validation
            )

grid.fit(X_train,y_train)
grid.best_params_

{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

In [18]:
#lakukan evaluasi dengan beberapa maetric di bawah ini
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      1983
           1       0.85      0.39      0.54       517

    accuracy                           0.86      2500
   macro avg       0.85      0.69      0.73      2500
weighted avg       0.86      0.86      0.84      2500


[[1947   36]
 [ 315  202]]

0.8596


## Model3
### Soal :
Model 3: DecisionTreeClassifier
Model ini menggunakan Decision Tree, yaitu metode machine learning yang mengambil keputusan dengan membentuk struktur pohon berdasarkan pemilihan fitur dan nilai terbaik untuk memisahkan data berlabel ke dalam kelas-kelas yang berbeda.

In [30]:
from sklearn.tree import DecisionTreeClassifier

#START CODE
model1 = DecisionTreeClassifier(random_state=42)
params = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}
#END CODE

#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= model1,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs = 10, # core cpu yang digunakan
             cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'min_samples_leaf': 4,
 'min_samples_split': 10}

In [31]:
#lakukan evaluasi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.94      0.91      1983
           1       0.69      0.49      0.57       517

    accuracy                           0.85      2500
   macro avg       0.78      0.72      0.74      2500
weighted avg       0.84      0.85      0.84      2500


[[1871  112]
 [ 265  252]]

0.8492


## Tarik Kesimpulan Model Mana yang terbaik beserta alasannya

Berdasarkan hasil evaluasi Random Forest Classifier, memiliki performa yang lebih baik dengan akurasi sekitar 85.96%. Alasan utama pemilihan ini adalah sebagai berikut:

Akurasi Tinggi: Model Random Forest memiliki akurasi lebih tinggi dibandingkan dengan dua model lainnya. Dalam konteks ini, akurasi merupakan metrik utama yang mengindikasikan seberapa baik model dapat mengklasifikasikan data dengan benar.

Performa Keseimbangan: Model Random Forest memiliki nilai presisi (precision) dan recall yang seimbang untuk kedua kelas (0 dan 1). Presisi mengukur seberapa baik model memprediksi positif, sedangkan recall mengukur seberapa baik model menangkap semua kasus positif yang sebenarnya. Model ini memberikan hasil yang baik untuk kedua metrik ini, menunjukkan kecenderungan tidak ada kecenderungan yang signifikan dalam memilih salah satu metrik di atas yang lain.

F1-Score yang Baik: F1-score adalah metrik gabungan dari presisi dan recall. Model Random Forest menunjukkan nilai F1-score yang baik untuk kedua kelas, menandakan keseimbangan yang baik antara presisi dan recall.

Pemrosesan Data Lebih Rumit: Random Forest mampu menangani hubungan yang kompleks dalam data karena menggunakan banyak decision tree. Ini membuatnya lebih efektif dalam menangani pola yang rumit dalam dataset dibandingkan dengan model lain yang lebih sederhana seperti Decision Tree.

Kemampuan untuk Menangani Overfitting: Random Forest memiliki teknik pengurangan overfitting bawaan seperti pembatasan kedalaman pohon dan jumlah minimum sampel di leaf node. Ini membantu mengurangi risiko overfitting pada model terutama ketika dataset memiliki banyak fitur.

Oleh karena itu, berdasarkan akurasi, keseimbangan antara presisi dan recall, serta kemampuan menangani kompleksitas dataset, model Random Forest Classifier dianggap yang terbaik untuk tugas klasifikasi pada dataset ini.
